In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint = "bigcode/starcoder"
device = "cuda" # for GPU usage or "cpu" for CPU usage

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# to save memory consider using fp16 or bf16 by specifying torch_dtype=torch.float16 for example
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map='auto', load_in_8bit=True)

2024-03-14 00:15:51.417834: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 00:15:51.417871: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 00:15:51.418843: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 00:15:51.423695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 00:15:51.969823: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
inputs = tokenizer.encode("def merge_sort():", return_tensors="pt").to(device)
outputs = model.generate(inputs)
# clean_up_tokenization_spaces=False prevents a tokenizer edge case which can result in spaces being removed around punctuation
print(tokenizer.decode(outputs[0], clean_up_tokenization_spaces=True))

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
checkpoint = "bigcode/starcoderbase-1b"

model = AutoModelForCausalLM.from_pretrained("checkpoints2/checkpoint-60", load_in_8bit=True, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(checkpoint, load_in_8bit=True, device_map='auto')

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_length=3000)


print( pipe("def hello():") )

In [6]:
!python -m torch.distributed.run --nproc_per_node 2 finetune/finetune.py   --model_path="bigcode/starcoderbase-1b"  --dataset_name="ArmelR/stack-exchange-instruction"  --subset="data/finetune"  --split="train"  --size_valid_set 2000  --streaming  --seq_length 4096  --max_steps 100  --batch_size 1  --input_column_name="prompt"  --output_column_name="target"  --gradient_accumulation_steps 16  --learning_rate 1e-4  --lr_scheduler_type="cosine"  --num_warmup_steps 10  --weight_decay 0.05  --output_dir="./checkpoints" --bf16

2024-03-14 02:19:59.956286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 02:19:59.956322: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 02:19:59.957285: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 02:19:59.961902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 02:20:00.557322: W tensorflow/compiler/tf2

In [8]:
import glob
from datasets import IterableDataset

def gen():
    prompts_list = glob.glob('dataset/train/prompts/*.txt')
    prompts_list.sort()
    ref_list = glob.glob('dataset/train/ref_scenarios/*.xosc')
    ref_list.sort()
    target_list = glob.glob('dataset/train/target_scenarios/*.xosc')
    target_list.sort()

    for i in range(len(ref_list)):
        with open(prompts_list[i], 'r') as prompt:
            with open(ref_list[i], 'r') as ref:
                with open(target_list[i], 'r') as target:
                    prompt = prompt.read()
                    ref = ref.read()
                    add = "<!-- " + prompt + " -->\n"
                    ref = add + ref
                    target = target.read()
                    yield {'prompt': ref, 'target': target}

ds = IterableDataset.from_generator(gen)

for example in ds:
    print(example)
    break

{'prompt': '<!-- would you add pedestrian close to hero -->\n<?xml version="1.0" ?>\n<OpenSCENARIO>\n  <FileHeader revMajor="1" revMinor="0" date="2020-03-24T12:00:00" description="CARLA:LaneChangeSimple" author=""/>\n  <ParameterDeclarations/>\n  <CatalogLocations/>\n  <RoadNetwork>\n    <LogicFile filepath="Town04"/>\n    <SceneGraphFile filepath=""/>\n  </RoadNetwork>\n  <Entities>\n    <ScenarioObject name="hero">\n      <Vehicle name="vehicle.tesla.model3" vehicleCategory="car">\n        <ParameterDeclarations/>\n        <Performance maxSpeed="69.444" maxAcceleration="200" maxDeceleration="10.0"/>\n        <BoundingBox>\n          <Center x="1.5" y="0.0" z="0.9"/>\n          <Dimensions width="2.1" length="4.5" height="1.8"/>\n        </BoundingBox>\n        <Axles>\n          <FrontAxle maxSteering="0.5" wheelDiameter="0.6" trackWidth="1.8" positionX="3.1" positionZ="0.3"/>\n          <RearAxle maxSteering="0.0" wheelDiameter="0.6" trackWidth="1.8" positionX="0.0" positionZ="0.3"